In [13]:
import numpy as np
import random

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(42)
random.seed(42)

In [14]:
from DataExploration import DataExploration
from DecisionTree import DecisionTree
from NeuralNetwork import DNN
from Boosting import Boosting
from KNN import KNN
from NeuralNetworkKeras import NN_Keras
from SVM import SVM


In [15]:
#need to shuffle to respect iid principal for when we split the data
de = DataExploration()

twitter_df = de.prepare_twitter_df()
speed_dating_df = de.prepare_speed_dating_df()

(22223, 10)


(19276, 10)
(41499, 10)


(8378, 44)


C:\Users\matan\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


removing empty values shape:
(6794, 44)


In [8]:
print(twitter_df.describe())
print(speed_dating_df.describe())

       num_following  num_followers     num_tweets  len_of_screen_name  \
count   41499.000000   4.149900e+04   41499.000000        41499.000000   
mean     1337.008603   1.490572e+03    1802.296754           10.763802   
std      4225.695112   2.541982e+04    5371.796703            2.754737   
min         0.000000   0.000000e+00       0.000000            2.000000   
25%        76.000000   4.000000e+01      49.000000            9.000000   
50%       285.000000   1.490000e+02     334.000000           11.000000   
75%      1119.000000   6.160000e+02    1511.000000           13.000000   
max    109388.000000   4.309347e+06  350453.000000           15.000000   

       len_of_profile_description  num_following_changes       is_spam  
count                41499.000000           41499.000000  41499.000000  
mean                    53.998530             878.826285      0.071014  
std                     51.940551             270.955164      0.997487  
min                      0.000000        

In [7]:
X_train, X_test, y_train, y_test, X_train_nn, X_validation_nn, y_train_nn, y_validation_nn = de.get_train_test_validation(
    twitter_df, 'twitter')

In [ ]:
# print(speed_dating_df.head(5))
# print(twitter_df.head(5))

In [ ]:
# data processing and investigation
data_twitter = twitter_df.drop('created_at', axis=1).drop('collected_at', axis=1).drop(
                'series_of_num_following', axis=1)
data_twitter['label'] = np.where(data_twitter['is_spam'] == 1, 'spam', 'not_spam')
data_twitter = data_twitter.drop('is_spam', axis=1)
pos_twitter = data_twitter.loc[data_twitter['label'] == 'spam'].drop('label', axis=1)
neg_twitter = data_twitter.loc[data_twitter['label'] == 'not_spam'].drop('label', axis=1)

create_scatterplot_matrix(data_twitter, "label", 'twitter_df')
plot_violin_distributions(data_twitter, title='Violin Plots of Twitter Features', dataset_name='twitter_df', label_column='label')
compare_counts_boxplots(positive_pd=pos_twitter, negative_pd=neg_twitter, title='Boxplots of Twitter Features', positive_label='spam', dataset_name='twitter_df')


In [ ]:
best_params = {'dt__alpha': -0.01, 'dt__max_depth': 9, 'dt__min_samples_leaf': 7}
dt_clf = DecisionTree(model_name='dt_wip1', dataset_name='spam_twitter', X_test=X_test, y_test=y_test, best_params=best_params)
dt_clf.train(X_train, y_train)
y_pred = dt_clf.predict(X_test)
dt_clf.evaluate_model(y_test=y_test, y_pred=y_pred)

#dt_clfr.get_model().best_estimator_.named_steps['dt'].tree_.max_depth


In [11]:
dt_best_params = dt_clf.get_best_params()
print(dt_best_params)
#print(dt_clf.feature_importance_)

NameError: name 'dt_clf' is not defined

In [7]:
# dnn = DNN(n_classes=2, model_name='dnn_wip_3', dataset_name='spam_twitter')
# dnn.train(
#     learning_rate=0.05,
#     steps=1000,
#     batch_size=30,
#     hidden_units=[100, 100],
#     training_examples=X_train_nn,
#     training_targets=y_train_nn,
#     validation_examples=X_validation_nn,
#     validation_targets=y_validation_nn)
# 
# y_pred = dnn.predict(X_test, y_test)

In [8]:
best_params = {'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 80}
# todo this is lower than pre pruning, figure out why
boosting_clf = Boosting(model_name='boosting_wip0', dataset_name='spam_twitter', X_test=X_test, y_test=y_test,
                        best_params=best_params)
boosting_clf.train(X_train, y_train)
y_pred = boosting_clf.predict(X_test)
boosting_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


[[3419  463]
 [ 522 3896]]
             precision    recall  f1-score   support

         -1       0.87      0.88      0.87      3882
          1       0.89      0.88      0.89      4418

avg / total       0.88      0.88      0.88      8300



In [9]:
boosting_best_params = boosting_clf.get_best_params()
print(boosting_best_params)

{'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 80}


In [10]:
best_params = {'knn__metric': 'manhattan', 'knn__n_neighbors': 10, 'knn__weights': 'distance'}
knn_clf = KNN(model_name='knn_wip0', dataset_name='spam_twitter', best_params=best_params)
knn_clf.train(X_train, y_train)
y_pred = knn_clf.predict(X_test)
knn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


[[3453  429]
 [ 531 3887]]
             precision    recall  f1-score   support

         -1       0.87      0.89      0.88      3882
          1       0.90      0.88      0.89      4418

avg / total       0.88      0.88      0.88      8300



In [11]:
knn_best_params = knn_clf.get_best_params()
print(knn_best_params)

{'knn__metric': 'manhattan', 'knn__n_neighbors': 10, 'knn__weights': 'distance'}


In [12]:
best_params = {'batch_size': 10, 'epochs': 1, 'layer_sizes': [12, 1], 'learning_rate': 0.0005,
               'loss': 'mean_squared_error'}

keras_nn_clf = NN_Keras(model_name='keras_nn_wip1', dataset_name='spam_twitter', best_params=best_params)
keras_nn_clf.train(X_train, y_train)
y_pred = keras_nn_clf.predict(X_test)
keras_nn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


ValueError: ('Could not interpret loss function identifier:', 0.0005)

In [13]:
keras_nn_best_params = keras_nn_clf.get_best_params()
print(keras_nn_best_params)

In [14]:
# keras_nn_clf = NN_Keras(model_name='keras_nn_optimizer_wip0', dataset_name='spam_twitter')
# keras_nn_clf.train(X_train, y_train, optimizer_tuning=True)
# y_pred = keras_nn_clf.predict(X_test)
# keras_nn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


In [15]:
# from sklearn.svm import SVC
# 
# linear_svm_clf = SVC(kernel='linear') #optimize kernel, regularization, gamma
# linear_svm_clf.fit(X_train, y_train)
# y_pred = linear_svm_clf.predict(X_test)

In [7]:
best_params = {'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}
svm_clf = SVM(model_name='svm_wip0', dataset_name='spam_twitter', best_params=best_params)
svm_clf.train(X_train, y_train)
y_pred = svm_clf.predict(X_test)
svm_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


Best parameter (CV score=0.860):
{'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}


[[3388  547]
 [ 596 3769]]
             precision    recall  f1-score   support

         -1       0.85      0.86      0.86      3935
          1       0.87      0.86      0.87      4365

avg / total       0.86      0.86      0.86      8300



In [9]:
svm_best_params = svm_clf.get_best_params()
print(svm_best_params) 
#{'svm__degree': 1, 'svm__kernel': 'rbf'}

{'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}


In [16]:
X_train, X_test, y_train, y_test, X_train_nn, X_validation_nn, y_train_nn, y_validation_nn = de.get_train_test_validation(
    speed_dating_df, 'speed_dating')



In [17]:
from sklearn.pipeline import Pipeline

from DecisionTree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier()

pipe = Pipeline(steps=[('scale', StandardScaler()),
                               ('dt', dt_model)])

pipe.fit(X_train, y_train)


TypeError: __init__() missing 2 required positional arguments: 'X_test' and 'y_test'